In [1]:
import json
f = open('train.json')
data = json.load(f)

In [2]:
data[100]

{'image': 'VG_100K_2/2415130.jpg',
 'answer': 6,
 'data_source': 'imported_genome',
 'question': 'How many cows are shown?',
 'image_id': 92415130,
 'question_id': 30092763}

In [3]:
import torch
import torch.nn as nn
import torchvision.models as models

In [4]:
resnet = models.resnet152(pretrained = True)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
newmodel = torch.nn.Sequential(*(list(resnet.children())[:-2]))
print(newmodel)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [7]:
x = torch.randn(1, 3, 224, 224)
output = newmodel(x)
output.shape

torch.Size([1, 2048, 7, 7])

In [8]:
net = nn.Conv2d(2048, 2048, kernel_size = 4, stride = (1, 1), bias = False)

In [9]:
resnet152 = nn.Sequential(newmodel, net)

In [10]:
output = resnet152(x)
output.shape

torch.Size([1, 2048, 4, 4])

In [11]:
import base64
import numpy as np
import csv
import sys
import zlib
import time
import mmap

csv.field_size_limit(sys.maxsize)
   
FIELDNAMES = ['image_id', 'image_w','image_h','num_boxes', 'boxes', 'features']
infile = 'trainval_resnet101_faster_rcnn_genome_36.tsv'


# Verify we can read a tsv
in_data = {}
c = 0
with open(infile, "r") as tsv_in_file:
    reader = csv.DictReader(tsv_in_file, delimiter='\t', fieldnames = FIELDNAMES)
    for item in reader:
        c += 1
        print(c)
        item['image_id'] = int(item['image_id'])
        item['image_h'] = int(item['image_h'])
        item['image_w'] = int(item['image_w'])   
        item['num_boxes'] = int(item['num_boxes'])
        for field in ['boxes', 'features']:
            item[field] = np.frombuffer(base64.b64decode(item[field]), 
                  dtype=np.float32).reshape((item['num_boxes'],-1))
        in_data[item['image_id']] = item
        if c == 2000:
            break


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000


In [16]:
for i in in_data:
    print(in_data[i])
    break

{'image_id': 150367, 'image_w': 640, 'image_h': 480, 'num_boxes': 36, 'boxes': array([[  0.        , 277.6827    , 583.3728    , 479.2       ],
       [389.56876   , 239.18001   , 475.52667   , 330.16315   ],
       [300.33197   , 265.5234    , 387.11652   , 347.883     ],
       [118.00602   , 138.60199   , 289.6468    , 235.06424   ],
       [  0.        ,   0.        , 252.79614   , 140.41837   ],
       [156.60852   , 352.786     , 344.48508   , 450.08588   ],
       [280.75128   , 146.12346   , 347.84033   , 265.48737   ],
       [386.7497    ,  44.484303  , 522.03094   , 154.81404   ],
       [338.34506   ,   0.        , 639.2       , 164.73306   ],
       [321.5712    , 358.26862   , 377.40723   , 448.2745    ],
       [279.20203   ,   0.        , 584.5702    , 132.85336   ],
       [263.4151    , 366.8589    , 314.9948    , 444.99268   ],
       [440.21368   , 156.93144   , 500.5992    , 193.9488    ],
       [  0.        ,   0.        , 608.7625    , 350.63208   ],
       [452

In [17]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [18]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [19]:
sentence_embeddings = sbert_model.encode("How many dogs are present in this array?")

In [25]:
sentence_embeddings.shape

(768,)

In [21]:
class tallyQA(nn.Module):
    def __init__(self):
        super(tallyQA, self).__init__()
        I_CNN = 2048
        Q_GRU_out = 768
        Q_embedding = 300
        LINsize = 1024
        Boxcoords = 16
        self.Ncls = 15
        
        layers_g1 = [ nn.Linear( 2*I_CNN + Q_GRU_out, LINsize),
                       nn.ReLU(inplace=True),
                       #nn.Dropout(0.5),
                       nn.Linear( LINsize, LINsize),
                       nn.ReLU(inplace=True),
                       #nn.Dropout(0.5),
                       nn.Linear(LINsize,LINsize),
                       nn.ReLU(inplace=True)]

        self.g1 = nn.Sequential(*layers_g1)
        self.g2 = nn.Sequential(*layers_g1)

        D = LINsize//2

        layers_f1 = [ nn.Linear(2 * LINsize,D),
                      nn.ReLU(inplace=True),
                      #nn.Dropout(0.5),
                      nn.Linear(D,self.Ncls),
                      nn.Softmax() ]


        self.gamma = nn.Sequential(*layers_f1)
    
    def forward(self, foreground, background, sentence_emb):
        sum1 = torch.zeros(1024)
        sum2 = torch.zeros(1024)
        for i in range(foreground.shape[0]):
            for j in range(foreground.shape[0]):
                if i != j:
                    f = torch.cat((foreground[i], foreground[j], sentence_emb), 0)
                    sum1 += self.g1(f)
                    
        for i in range(background.shape[0]):
            for j in range(foreground.shape[0]):
                t = torch.cat((foreground[j], background[i], sentence_emb), 0)
                sum2 += self.g2(t)
                
        return self.gamma(torch.cat((sum1, sum2), 0))

In [22]:
model = tallyQA()

In [23]:
f = torch.zeros((32, 2048))
b = torch.zeros((16, 2048))
s = torch.zeros((768))

In [24]:
t = model(f, b, s)

/home/vinnni002/.local/lib/python3.8/site-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [11]:
new_data = []
c = 0
for i in data:
    new_data.append(i)
    c += 1
    if c == 100:
        break

In [12]:
new_data

[{'image': 'VG_100K/4.jpg',
  'answer': 2,
  'data_source': 'imported_genome',
  'question': 'How many chairs are pictured?',
  'image_id': 90000004,
  'question_id': 30092475},
 {'image': 'VG_100K/4.jpg',
  'answer': 2,
  'data_source': 'imported_genome',
  'question': 'How many chairs are at the table?',
  'image_id': 90000004,
  'question_id': 30092476},
 {'image': 'VG_100K/4.jpg',
  'answer': 1,
  'data_source': 'imported_genome',
  'question': 'How many things are purple?',
  'image_id': 90000004,
  'question_id': 30092477},
 {'image': 'VG_100K/4.jpg',
  'answer': 2,
  'data_source': 'imported_genome',
  'question': 'How many curtains are there?',
  'image_id': 90000004,
  'question_id': 30092478},
 {'image': 'VG_100K/4.jpg',
  'answer': 2,
  'data_source': 'imported_genome',
  'question': 'How many chairs?',
  'image_id': 90000004,
  'question_id': 30092479},
 {'image': 'VG_100K/4.jpg',
  'answer': 1,
  'data_source': 'imported_genome',
  'question': 'How many pillows are on the 